In [141]:
import csv
import numpy as np
import math
import random
from geneticalgorithm2 import geneticalgorithm2 as ga 

In [2]:
interactions = []
#import all the interaction matricies created beforehand and add to array (flattened versions of the matrices)
for i in range(50):
    for j in range(50):
        mat = np.loadtxt(open("interactionMats/"+str(i)+"_"+str(j)+".csv", "rb"), delimiter=",", skiprows=0)
        arr = np.array(mat).flatten()

        interactions.append(arr)

In [3]:
def fitness_func(solution):
    #add all the interaction values
    wake = 0
    ones = []

    for i in range(2500):
        if solution[i] == 1:
            ones.append(i)
    if len(ones) > 50:
        return 1000000

    for i in ones:
        tot = 0
        for j in ones:
            tot += interactions[i][j]
        wake += tot
    return wake

In [4]:
def inversion_mutation(bitstring,prob):
    if np.random.rand() < prob:
        s = int(np.random.rand() * len(bitstring) )
        e = len(bitstring) - int((np.random.rand() * min(20,(len(bitstring) - s)))) - 1

        new_string =np.flip(np.copy(bitstring[s:e+1]))

        for i in range(s,e+1):

            bitstring[i] = new_string[i-s]
    return bitstring

In [129]:
def scramble_mutation(bitstring,prob):
    if np.random.rand() < prob:
        s = int(np.random.rand() * len(bitstring) )
        e = len(bitstring) - int((np.random.rand() * min(20,(len(bitstring) - s)))) - 1

        new_string =np.copy(bitstring[s:e+1])
        np.random.shuffle(new_string)
        for i in range(s,e+1):
            bitstring[i] = new_string[i-s]
    return bitstring

In [6]:
def swap_mutation(bitstring,prob):
    lens = len(bitstring)

    if np.random.rand() < prob:
        i = int(np.random.rand() * lens)

        # flip the bit and one with one on opposite end
        curr = np.copy(bitstring[i])
        prev = np.copy(bitstring[lens - i - 1])
        bitstring[i] = prev
        bitstring[lens - i - 1] = curr

    return bitstring

In [95]:
def rand_mutation(bitstring, prob):
	lens = len(bitstring)

		
	if np.random.rand() < prob:
		i = int(np.random.rand() * lens)

		# flip the bit and one with opposite value (to keep same number of turbines)
		prev = np.copy(bitstring[i])
		opp = np.where(bitstring == (1-prev))[0]
		bitstring[i] = 1 - np.copy(prev)

		bitstring[  opp[int(np.random.rand()*len(opp))]  ] = np.copy(prev)


	return bitstring

In [139]:
def make_arr(ones: np.ndarray):
    arr = [0] * 2500
    for i in ones:
        arr[i] = 1
    return np.array(arr)

In [16]:
def uniform_crossover(x: np.ndarray, y: np.ndarray):
    #randomly selects 1's from either sides
    x1 = np.where(x == 1)[0]
    y1 = np.where(y == 1)[0]

    ofs1 = []
    ofs2 = []
    
    all = list(set(x1) | set(y1))

    # ofs1 = random.randrange(len(test_list))
    # ofs1 = all[rand_idx]

    ofs1 = make_arr(ofs1)
    ofs2 = make_arr(ofs2)    
    return ofs1, ofs2

In [140]:
make_arr(np.array([3,4,15,20]))

array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0])